In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import entropy
import random
import copy
from tqdm import tqdm
import pickle
from datetime import datetime

In [ ]:
import sys, os
sys.path.insert(0, os.path.dirname(os.path.abspath('../../main_code')))
from main_code.neal_algorithm_3 import algorithm_3, save_data, load_data
from main_code.partitions_analysis import *
from main_code.data_generation import generate_gaussian_mixture
from main_code.visualization import *
from main_code.integrals_without_covariates import *

# ASSUMPTION:
compute_lamb_0 definisce uno degli hyperpatamemtri da cambiare 

In [3]:
# These functions change based on assumption on the prior parameters of the model
def compute_mu_0(Y):
    return np.mean(Y, axis=0)   # default to mean of data, reasonable assumption

def compute_nu_0(D):
    #! changed to D+3 from D+1 as suggested in last meeting
    return D + 3    # default to D, potentially change this

def compute_inv_scale_mat_0(D):
    return np.eye(D)    # default to identity, potentially change this

def compute_lamb_0(Y):
    #! changed to a value smaller then 1 as suggested in last meeting
    return 0.75    # default to 1, potentially change this

# SYNTHETIC DATA

In [ ]:
# Sample syntethic
np.random.seed(42)  # for reproducibility
random.seed(42)

# Y
means = [[0, 0], [2, -5], [-5, -5]]
covariances = [np.eye(2), 2 * np.eye(2), 0.5 * np.eye(2)]
n_samples = [50, 50, 50]

Y, labels = generate_gaussian_mixture(means, covariances, n_samples)

plot_clusters(Y, labels, "Synthetic Data from a Mixture of Gaussians")


# X
means_X = [[0, 0], [-10, -10], [10, 10]]
covariances_X = [0.5 * np.eye(2), 0.5 * np.eye(2), 0.5 * np.eye(2)]

X, _ = generate_gaussian_mixture(means_X, covariances_X, n_samples)

plot_clusters(X, labels, "Synthetic Covariates from a Mixture of Gaussians")

# Simulazione
alpha è uno degli hyperparametri da cambiare.

In [ ]:
n_steps = 6000
burn_in = 1000
alpha = 1

In [ ]:
# Trying our clustering algorithm without covariates
history, entropies = algorithm_3(n_steps,
                      Y,
                      X,
                      integral_func_1,
                      integral_func_2,
                      alpha=alpha,
                      compute_nu_0=compute_nu_0,
                      compute_lamb_0=compute_lamb_0,
                      visualize_entropy=True,
                      lambda_penalty=0.0)

S = sampled_sim_matrix(history[burn_in:], len(Y))

optimal_clustering, optimal_loss = find_optimal_clustering_binder_loss(history[burn_in:], S, alpha=1.0, beta=1.0)

print("The loss is equal to: ", optimal_loss)

plot_clusters(Y, optimal_clustering, "Result of Posterior Point Estimate")